In [ ]:
%matplotlib inline
%reload_ext autoread
%autoread 2

Importamos las bibliotecas que se van a necesitar (fastai, pytorch)


In [ ]:
import fastai as fai
import fastai.vision as fv
from pathlib import Path 

Tenemos una carpeta con subcarpetas de las clasificaciones de flores y en cada subcarpeta hay imagenes de la especie de flores respectivas a cada subcarpeta (Sé que no las ocupo porque los datos ya están descargados, pero los pongo para que no se me olvide)

In [ ]:
#!wget https://my.pcloud.com/publink/show?code=XZiwtw7Z15TY80dPm4b3KwmNcf1ChfphgM5y
#!tar xzf flowers.tar.gz

In [ ]:
!ls flowers 

In [ ]:
!ls flowers/train

In [ ]:
Flores = Path("flowers")

In [ ]:
transformaciones = fv.get_transforms(flip_vert=True, max_rotate=360,
                         max_zoom=1.25, max_lighting=0.3)

In [ ]:
datos = (fv.ImageDataBunch
        .from_folder(Flores, ds_tfms=transformaciones, size=224, bs=64)
        .normalize(fv.imagenet_stats))

Mostramos las imagenes

In [ ]:
datos.show_batch(rows=3)

ENTRENAMIENTO
En esta parte vamos a crear algo llamado learner, que se va a encargar de hacer el aprendizaje con base al entrenamiento de este.
resnet18 - es la arquitectura del learner (y es todo lo que sé)
En metrics le pedimos que nos muestre el acuracy de la clasificación y el error del mismo

In [ ]:
learner = fv.create_cnn(datos, fv.models.resnet18, 
                        metrics=[fai.metrics.accuracy, fai.metrics.error_rate], wd=0.1)

Ahora se usa la función fit_one_cycle para entrenar la red neuronal de una manera rápida (supongo que solo hace entrenamiento en las salidas o algo así, no sé...)

In [ ]:
learner.fit_one_cycle(1)

In [ ]:
learner.fit_one_cycle(4)

Como no sabes cual es la taza de aprendizaje que debemos utilizar, vamos a graficar los errores que comete la red neuronal con diferentes tazas de aprendizaje

In [ ]:
learner.lr_find(); learner.recorder.plot()

Como podemos ver, la taza de aprendizaje con menos "errores" es de 1e-2
Así que usaremos eso para realizar un nuevo entrenamiento del learner

In [ ]:
learner.fit_one_cycle(4, max_lr = 1e-2)

Y salvamos el entrenamiento del learner

In [ ]:
learner.save('stage1')

Ahora ploteamos los errores en los sets de train y validation

In [ ]:
learner.recorder.plot_losses()

Esto no recuedo para qué sirve, pero parece que grafica el error y accuracy del learner

In [ ]:
learner.recorder.plot_lr(show_moms=True)

Como expliqué anteriormente, solo hicimos entrenamiento en las últimas capas porque el learner ya venía con una arquitectura entrenada, así que se tienen que "descongelar" para poder entrenar la red completamente

In [ ]:
learner.unfreeze()

Con este comando se va a buscar la mejor taza de aprendizaje ya con la red completa 

In [ ]:
learner.lr_find(); learner.recorder.plot()

Antes de entrenar toda la red se van a modificar las primeras capas. y se agrega una taza de aprendizaje de entre 1e-5 a 1e-4 ya que ahí se vió un menor error

In [ ]:
learner.fit_one_cycle(5, max_lr=slice(1e-5,1e-4))

Y salvamos los resultados 

In [ ]:
learner.save('stage2')

Ahora vemos el resultado de la clasificación en el conjunto de validation

In [ ]:
learner.show_results(ds_type=fai.DatasetType.Valid, rows=3)

Checamos el accuracy

In [ ]:
fai.metrics.accuracy(*learner.TTA())

Y finalmente vemos los errores de clasificación

In [ ]:
interpretacion = learner.interpret()

In [ ]:
interpretacion.plot_top_losses(9)